In [1]:
import pandas as pd
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
from scipy.signal import butter, lfilter

c:\Users\mahim\.conda\envs\DS203_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def invert_MFCC_to_audio(mfcc_coefficients, sr=44100, n_iter=64):
    """
    Invert MFCC coefficients back to audio signal.
    
    Parameters:
    mfcc_coefficients (numpy.ndarray): The MFCC coefficients.
    sr (int): The sampling rate of the audio.
    n_iter (int): Number of iterations for the Griffin-Lim algorithm.
    
    Returns:
    numpy.ndarray: The reconstructed audio signal.
    """
    mfcc_coefficients = np.nan_to_num(mfcc_coefficients)
    mel_spectrogram = librosa.feature.inverse.mfcc_to_mel(mfcc_coefficients)
    mel_spectrogram = np.nan_to_num(mel_spectrogram)
    audio_signal = librosa.feature.inverse.mel_to_audio(mel_spectrogram, sr=sr, n_iter=n_iter)
    audio_signal = np.nan_to_num(audio_signal)
    
    return audio_signal
def normalize_audio(audio_signal):
    
    
    return librosa.util.normalize(audio_signal)

def butter_highpass(cutoff, fs, order=8):
    
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def highpass_filter(audio_signal, cutoff=150, fs=44100, order=8):
    
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, audio_signal)
    return y

# Read the CSV file into a pandas DataFrame
data = pd.read_csv('05-MFCC.csv', header=None)

# Convert the single row DataFrame to a numpy array
mfccs = data.values.flatten()

# Reshape the MFCC coefficients to the correct 2D array format
# Assuming the original MFCCs had 20 coefficients per frame
n_mfcc = 20
mfccs = mfccs.reshape((n_mfcc, -1))

# Invert MFCC to audio with increased Griffin-Lim iterations
audio_signal = invert_MFCC_to_audio(mfccs, sr=44100, n_iter=64)

# Apply noise reduction without the use_tensorflow parameter
audio_signal = nr.reduce_noise(y=audio_signal, sr=44100, prop_decrease=0.9)

# Normalize the audio signal
audio_signal = normalize_audio(audio_signal)

# Apply a higher-order high-pass filter to remove low-frequency noise
audio_signal = highpass_filter(audio_signal, cutoff=150, fs=44100, order=8)

# Store the audio signal to a file
sf.write('5_refined_clear.wav', audio_signal, 44100)

print("Audio signal saved successfully as '5_refined_clear.wav'.")

Audio signal saved successfully as '5_refined_clear.wav'.
